In [ ]:
import torch
import sklearn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn
import matplotlib.pyplot as plt
import torch


## PLAN!!!

### Things to do:
- Could investigate the dataset(s), see if there is any pattern from plotting etc. (Seems quite dead)
- Create models from list below:

#### Models to consider
- Polynomial Regression using Basis Expansion (Scikit-learn mostly)
- Fully connected Neural Network (Probably PyTorch)
- Investigate how to use ChatGPT or something
- Support Vector Regression (Scikit-learn SVR)
- Ensemble Method: Random Forest Regressor (Also scikit-learn probably)

#### Considerations:
- For each model, there are most likely hyperparameters that can be tuned.
- E.g. polynomial regression can use L2 or L1 (or both) regularization to avoid overfitting, this has a parameter that can be tuned
- Haven't looked into the others but probably just ask ChatGPT
- Can use k-fold cross validation to validate hyper-parameters (or just use validation test set, k-fold would be more robust though)
- Trial and error to find a decent starting point then search in the parameter space around it using for loops (GridSearch I guess)
- HOPE IT WORKS!

In [ ]:
# Data investigation
def load_data(n_to_load):
    # Possible n's include: 9, 12, 15, 18, 24, 30, 45.
    X = np.load('Datasets/kryptonite-%s-X.npy'%(n_to_load))
    y = np.load('Datasets/kryptonite-%s-y.npy'%(n_to_load))

    for n in range(X.shape[1]):
        print(X[:, n].shape)
        y = np.arange(X.shape[0])
        plt.scatter(y, X[:, n])
        plt.show()
        print(X.shape)
        print(y.shape)

    return X, y

load_data(9)

## Polynomial regression attempt

Firstly start with polynomials of degree 2 (up to 3) since for datasets n=45, having larger order polynomials will probably cause the model to overfit.

Add in regularization:
- Could investigate using Ridge Regression (L2) or Lasso Regression (L1) individually and tune hyperparameters there.
- Could investigate ElasticNet which combines both L2 and L1 regularisation 

In [ ]:
def polynomial_main(n, e):
    pass



In [ ]:
# Load data
def main(n, e):
    X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
    y = np.load('Datasets/kryptonite-%s-y.npy'%(n)) 

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)  # 60% training
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% validation, 20% test

    main()
    # Ideas to try:
    # Polynomial Regression with Basis Expansion (?)
    # Fully Connected Neural Network
    # Support Vector Regression
    # Random Forest Regression
    # GPT-4?